# Audio key estimation of digital music with CNNs

## Data Preprocessing

### Million Song Dataset
- utilized to select appropriate song samples
- holds information about key and mode per song (used as targets)

Juypter Notebook <a href='./00.hlp/msd/msd.ipynb'>msd</a>

outputs: csv file *songs_conf=75_tracks_filt.csv*, which holds all songs with key confidence and mode confidence > 0.75

In [8]:
# LIST SELECTED SONGS
import os
import pandas as pd
from IPython.display import display

selsongsfile = os.path.join ('00.hlp', 'msd', 'songs_conf=75_tracks_filt.csv')
selsongs = pd.read_csv (selsongsfile, header=0, index_col=0)
display (selsongs.head ())
print ('[i] number of records:', len (selsongs))

,key,key_confidence,mode,mode_confidence,track_id,song_id,artist_name,song_title
0,7,0.896,1,0.852,TRMMMGL128F92FD6AB,SOHSSPG12A8C144BE0,Clifford T. Ward,Mad About You
1,10,0.882,1,0.799,TRMMWMT128F9320CFD,SOFSJXP12AB017E676,Katie Armiger,I Just Want To Be With You This Christmas
2,2,0.966,1,0.990,TRMMWZE12903D0EB43,SOEBDKM12AC4686D3A,Kultiration,Bakslag
3,0,1.000,1,0.905,TRMMWMT128F9345B1B,SOXAGQZ12AB018A089,Andrea Klas,Ride
4,2,1.000,1,0.991,TRMMWJM128F4227255,SOOWUDY12A6D4FA0B0,Archie Roach,Summer Of My Life


[i] number of records: 47913


In [12]:
# LOAD DATASET
import os
import numpy as np
from sklearn import datasets

PARAM_RND_STATE = 42

container_path = os.path.join ('src_audio')
load_content = False
# keys: C, C#, D, D#, E, F, F#, G, G#, A, A#, B
# https://commons.wikimedia.org/wiki/File:Frequencies_of_the_audible_range_on_a_twelve_and_eight_equal_tempered_scale.jpg
description = ['key C, mode minor', 'key C, mode major',
               'key C#, mode minor', 'key C#, mode major',
               'key D, mode minor', 'key D, mode major',
               'key D#, mode minor', 'key D#, mode major',
               'key E, mode minor', 'key E, mode major',
               'key F, mode minor', 'key F, mode major',
               'key F#, mode minor', 'key F#, mode major',
               'key G, mode minor', 'key G, mode major',
               'key G#, mode minor', 'key G#, mode major',
               'key A, mode minor', 'key A, mode major',
               'key A#, mode minor', 'key A#, mode major',
               'key B, mode minor', 'key B, mode major']

data = datasets.load_files (container_path=container_path,
                            description=description,
                            load_content=load_content,
                            random_state=PARAM_RND_STATE)

In [19]:
#print (data['target_names'], data['DESCR'])
#print (data['filenames'].size)

#unique_targets = list (set (data['target']))
#for i in range (len(unique_targets)):
#    print (data['filenames'][np.where (data['target']==unique_targets[i])], end='\n\n')

In [68]:
# get filenames
filenames = list (os.path.basename (filepath) for filepath in data['filenames'])
usedsongs_track_id = list (os.path.splitext (fn)[0] for fn in filenames)
usedsongs = selsongs.query ('track_id in @usedsongs_track_id')

In [85]:
display (usedsongs.sort_values (by=['key', 'mode'], axis=0))
print ('[i] min of: key_confidence =', usedsongs['key_confidence'].min (), ',', \
       'mode_confidence =', usedsongs['mode_confidence'].min ())

,key,key_confidence,mode,mode_confidence,track_id,song_id,artist_name,song_title
606,0,1.000,0,0.861,TRMQFWW128F1465DC5,SOVJPKX12A6D4F7362,Eternal,Who Are You
833,0,1.000,0,0.777,TRMAZNT12903CDF7A1,SOIWGJG12A58A7D278,Linea 77,Sleepless
929,0,1.000,0,0.877,TRMNUWW12903CE9B9F,SOHXVJO12A58A79A6B,Mekkanikka,Flame It
1988,0,1.000,0,0.962,TRWWUKN128F932A086,SOXSQMS12AB0181DB3,Judy Dyble,C'est La Vie
10070,0,1.000,0,0.849,TRRTCMY128F9326B32,SOYGPIR12AB018646A,The Levon Helm Band,A Fool In Love
18761,0,1.000,0,0.948,TRIRLUR128F9301274,SOVBANL12A8C13F2DD,Dirtmusic,Unknowable
27801,0,1.000,0,0.853,TRUGDTE128F14934A9,SONTQAC12A6D4FB232,Junior Kelly,Blaze
29273,0,1.000,0,0.889,TRUXMRH128F92F7A43,SOGTREL12AB0181DC8,Blues Company,Dark Day
46303,0,1.000,0,0.938,TRYGKEI12903D08088,SOLSAGB12AC468A4C4,Delegation,Darlin'
47391,0,1.000,0,0.830,TRYJAMS128F427EEC9,SOBTZUS12A8C13E2AC,Chandeen,Clean The Traces


[i] min of: key_confidence = 0.809 , mode_confidence = 0.777
